### Uczenie początkowe

Dane zostały przygotowane zgodnie z założeniami na w [instrukcji reprodukcji](https://github.com/ProjektCustomVisionKL/projekt-custom-vision-kl#instrukcje-reprodukcji), gdzie opisane są wymagane instalacje oraz rozmieszczenie plików i katalogów.

Z uwagi na czas trwania kolejnych iteracji w pierwszej fazie uczenia wykonanych zostało 10 iteracji. Jako, że użyty model był wstępnie przetrenowanym modelem YOLOv3, użyta została funkcja `setModelTypeAsYOLOv3`.

In [2]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="data")
trainer.setTrainConfig(object_names_array=["metal", "plastic", "glass"], batch_size=8, num_experiments=10, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.68
Anchor Boxes generated.
Detection configuration saved in  data/json/detection_config.json
Evaluating over 162 samples taken from data/validation
Training over 476 samples  given at data/train
Training on: 	['glass', 'metal', 'plastic']
Training with Batch Size:  8
Number of Training Samples:  476
Number of Validation Samples:  162
Number of Experiments:  10
Training with transfer learning from pretrained Model


/home/dlubera/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/dlubera/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/10
480/480 [==============================] - 3276s 7s/step - loss: 82.5083 - yolo_layer_loss: 9.9438 - yolo_layer_1_loss: 20.2975 - yolo_layer_2_loss: 40.8128 - val_loss: 40.2648 - val_yolo_layer_loss: 0.4663 - val_yolo_layer_1_loss: 8.5795 - val_yolo_layer_2_loss: 21.9052
Epoch 2/10
480/480 [==============================] - 3229s 7s/step - loss: 40.7164 - yolo_layer_loss: 5.4302 - yolo_layer_1_loss: 10.3312 - yolo_layer_2_loss: 16.7229 - val_loss: 34.5691 - val_yolo_layer_loss: 0.8266 - val_yolo_layer_1_loss: 8.1414 - val_yolo_layer_2_loss: 19.3945
Epoch 3/10
480/480 [==============================] - 3180s 7s/step - loss: 35.9190 - yolo_layer_loss: 4.8264 - yolo_layer_1_loss: 9.4424 - yolo_layer_2_loss: 15.7802 - val_loss: 32.4014 - val_yolo_layer_loss: 0.7090 - val_yolo_layer_1_loss: 8.0653 - val_yolo_layer_2_loss: 18.4707
Epoch 4/10
480/480 [==============================] - 3048s 6s/step - loss: 33.6060 - yolo_layer_loss: 4.5811 - yolo_layer_1_loss: 8.6114 - yolo_layer_2

Na podstawie kodu widać stopniowe wolną ale stopniową poprawnę jakości modelu (loss maleje).

### Sprawdzenie jakości

Korzystają z funckji `evaluateModel` można sprawdzić jakość wykrywania w każdej z iteracji modelu.

In [5]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="data")
metrics = trainer.evaluateModel(model_path="data/models", json_path="data/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

Starting Model evaluation....
Evaluating over 162 samples taken from data/validation
Training over 476 samples  given at data/train
Model File:  data/models/detection_model-ex-001--loss-0055.955.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0000
metal: 0.0000
plastic: 0.0000
mAP: 0.0000
Model File:  data/models/detection_model-ex-002--loss-0038.880.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0016
metal: 0.0042
plastic: 0.0051
mAP: 0.0036
Model File:  data/models/detection_model-ex-003--loss-0035.620.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0032
metal: 0.0156
plastic: 0.0345
mAP: 0.0177
Model File:  data/models/detection_model-ex-004--loss-0033.748.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glas

Informacje podane wyżej jak np: `metal: 0.1430` oznaczają precyzję dla konkretnej etykiety. Jak widać jakośc nie jest zachwycająca.

### Detekcja

Korzystając z klasy `CustomObjectDetection` użyliśmy stworzonego modelu po 10 iteracji i przetestowaliśmy wykrywanie. Obszar wykrycia był poprawy, jednakże minusem było to, że w tym samym miejscu wykryło zarówno metal jak i plastik (plastik został wykryty z większym prawdopodobieństwem), a faktycznie był tam sam plastik.

In [19]:
from imageai.Detection.Custom import CustomObjectDetection
import os

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("data/models/detection_model-ex-010--loss-0030.442.h5")
detector.setJsonPath("data/json/detection_config.json")
detector.loadModel()

In [26]:
detections, extracted_objects_array = detector.detectObjectsFromImage(input_image="data/test/46939963335_3c4167e593_z.jpg", output_image_path="detected.jpg", extract_detected_objects=True)

detections

[{'name': 'metal',
  'percentage_probability': 58.424848318099976,
  'box_points': [146, 384, 231, 407]},
 {'name': 'plastic',
  'percentage_probability': 75.00083446502686,
  'box_points': [146, 384, 231, 407]}]

### Douczanie

Aby douczyć model skorzystaliśmy z tych samych funkcji co przy pierwszej fazie uczenia, jednakże jako pretrenowany model został użyty model stworzony w 10 iteracji fazy uczenia.

In [1]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="data")
trainer.setTrainConfig(object_names_array=["metal", "plastic", "glass"], batch_size=8, num_experiments=8, train_from_pretrained_model="detection_model-ex-010--loss-0030.442.h5")
trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.68
Anchor Boxes generated.
Detection configuration saved in  data/json/detection_config.json
Evaluating over 162 samples taken from data/validation
Training over 476 samples  given at data/train
Training on: 	['glass', 'metal', 'plastic']
Training with Batch Size:  8
Number of Training Samples:  476
Number of Validation Samples:  162
Number of Experiments:  8
Training with transfer learning from pretrained Model


/home/dlubera/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


/home/dlubera/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/8
480/480 [==============================] - 2998s 6s/step - loss: 29.1350 - yolo_layer_loss: 4.7929 - yolo_layer_1_loss: 7.4442 - yolo_layer_2_loss: 13.1638 - val_loss: 27.6319 - val_yolo_layer_loss: 0.7971 - val_yolo_layer_1_loss: 7.5850 - val_yolo_layer_2_loss: 15.5732
Epoch 2/8
480/480 [==============================] - 2779s 6s/step - loss: 28.4966 - yolo_layer_loss: 4.7846 - yolo_layer_1_loss: 6.9157 - yolo_layer_2_loss: 13.1367 - val_loss: 27.1584 - val_yolo_layer_loss: 0.5992 - val_yolo_layer_1_loss: 6.8604 - val_yolo_layer_2_loss: 16.0838
Epoch 3/8
480/480 [==============================] - 3157s 7s/step - loss: 29.3163 - yolo_layer_loss: 5.4352 - yolo_layer_1_loss: 7.6095 - yolo_layer_2_loss: 12.6724 - val_loss: 26.7743 - val_yolo_layer_loss: 1.0131 - val_yolo_layer_1_loss: 7.2367 - val_yolo_layer_2_loss: 14.9628
Epoch 4/8
480/480 [==============================] - 2890s 6s/step - loss: 27.7025 - yolo_layer_loss: 4.8039 - yolo_layer_1_loss: 6.6992 - yolo_layer_2_loss:

Tutaj również widać wolną ale ciągłą poprawę modelu.

### Sprawdzanie jakości po douczeniu

W wynikach sprawdzania widać, że większość z iteracji poprawiała precyzję modelu i korzystając z kolejnych iteracji model mógłby działać jeszcze lepiej, jednakże ze względu na czas obliczeń musieliśmy zadowolić się modelem stworzonym w ostatniej iteracji. Podany model został użyty do stworzenia API wykrywającego obiekty.

In [1]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="data")
metrics = trainer.evaluateModel(model_path="data/models", json_path="data/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

Starting Model evaluation....
Evaluating over 162 samples taken from data/validation
Training over 476 samples  given at data/train
Model File:  data/models/detection_model-ex-001--loss-0029.354.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0234
metal: 0.1528
plastic: 0.1950
mAP: 0.1237
Model File:  data/models/detection_model-ex-002--loss-0028.724.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0151
metal: 0.1804
plastic: 0.1480
mAP: 0.1145
Model File:  data/models/detection_model-ex-003--loss-0035.620.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glass: 0.0032
metal: 0.0156
plastic: 0.0345
mAP: 0.0178
Model File:  data/models/detection_model-ex-004--loss-0033.748.h5 

Evaluation samples:  162
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
glas